In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# creating virtual env
!pip install -q condacolab
import condacolab
condacolab.install()
!conda --version
!which conda
!conda create --name envn python=3.7

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...
conda 23.11.0
/usr/local/bin/conda
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/envn

  added / updated specs:
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    libgcc-ng-13.2.0           |       h807b86a_5         752 KB  conda-forge
    libgomp-13.2.0             |       h807b86a_5 

In [1]:
# testing the python version
%%bash
source activate envn
python


import sys
print(sys.version)

3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:21) 
[GCC 9.4.0]


In [2]:
# installing requirements in conda environment
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project
pip install -r requirements_new.txt
python


import pandas as pd
print(pd.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.9/191.9 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.1 MB/s eta 0:00:00
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

## 1. Training a model with only sequence information (ESM-1b):

#### (a) Hyperparameter optimization:

In [ ]:
'''%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python


from sklearn.metrics import r2_score, mean_squared_error
from hyperopt import fmin, tpe, rand, hp, Trials
from scipy import stats
from scipy.optimize import minimize
import numpy as np
import pickle
import pandas as pd
from os.path import join
import xgboost as xgb
import warnings


## Loading training and test data:
train_indices = list(
    np.load(
        join("..", "..", "data", "kcat_data", "splits", "CV_train_indices.npy"),
        allow_pickle=True,
    )
)
test_indices = list(
    np.load(
        join("..", "..", "data", "kcat_data", "splits", "CV_test_indices.npy"),
        allow_pickle=True,
    )
)


data_train = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl"))
data_test = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl"))


data_train.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
data_test.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)



train_ESM1b = np.array(list(data_train["ESM1b"]))
train_X = train_ESM1b
train_Y = np.array(list(data_train["log10_kcat"]))

test_ESM1b = np.array(list(data_test["ESM1b"]))
test_X = test_ESM1b
test_Y = np.array(list(data_test["log10_kcat"]))


def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    print("R2 score: ", R2)
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn=cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 3, trials=trials)

print((best))

# for train_indice, test_indice in zip(train_indices, test_indices):
#     print(len(train_indice), len(test_indice))
#     print(type(train_indice), type(test_indice))
#     print(train_indice)
'''

R2 score: 
[0.29000268650057703, 0.26266027745322174, 0.2628877807027552, 0.26778315057957514, 0.3004563477070774]
R2 score: 
[0.2615145740893142, 0.21452132377594235, 0.22699209221764238, 0.14388907790226801, 0.1784535397229029]
R2 score: 
[0.3092307419728141, 0.31958664352804367, 0.31321625606695236, 0.24651225897998863, 0.33721263694900094]
100%|██████████| 3/3 [00:21<00:00,  7.33s/trial, best loss: -0.30515170749935994]
{'learning_rate': 0.1689725592926287, 'max_delta_step': 2.9992069924682503, 'max_depth': 4.482067672390703, 'min_child_weight': 6.66251654614674, 'num_rounds': 51.219215908837775, 'reg_alpha': 3.811237461892115, 'reg_lambda': 0.9617522719421612}


In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python


from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
  data_train["difference_fp"],
  data_train["ESM1b_ts"]
  ]
test_X = [
  data_test["difference_fp"],
  data_test["ESM1b_ts"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


find_best_params(train_X, train_Y, test_X, test_Y, max_evals=3)

R2 score: 
0.10459828378088516
R2 score: 
-0.04026632933275136
R2 score: 
0.31481239697554225
100%|██████████| 3/3 [03:13<00:00, 64.61s/trial, best loss: -0.31481239697554225]
Best params:  {'learning_rate': 0.2052435648033681, 'max_delta_step': 2.1911893271725678, 'max_depth': 11.595220364403353, 'min_child_weight': 9.56979739893249, 'num_rounds': 167.75448394398296, 'reg_alpha': 3.635932090723124, 'reg_lambda': 2.5380197833025457}


#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl"))
data_test = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl"))


data_train.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
data_test.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)


train_X = [data_train["ESM1b"]]
test_X = [data_test["ESM1b"]]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.051447544749765035,
    "max_delta_step": 2.956459783615207,
    "max_depth": 5.034202474908222,
    "min_child_weight": 7.457989829577018,
    "num_rounds": 400.50601395689256,
    "reg_alpha": 1.0858835704466614,
    "reg_lambda": 1.1385559144302175,
}

model_name = "esm1b"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)


[
  "esm1bts_esm1b_combined_all_feats.h5", # 38
  "esm1bts_all_new_feats.h5", # 41
  esm1bts_drfp_traid_combined, # 40
  esm1bts_drfp_paac_combined, #40
  esm1bts_drfp_ctd_combined, # 39
  esm1bts_drfp_geary_auto_combined, # 37
  esm1bts_drfp_moran_auto_combined, # 40
  esm1bts_drfp_moreau_broto_auto_combined, # 38
  esm1bts_drfp_dp_composition_combined, # 37
  esm1bts_drfp_aa_composition, # 41
  esm1bts_drfp_chem, # 42,
  esm1bts_drfp_maccs, # 40,
  esm1bts_diff, # 42
  esm1bts_drfp,
  structural_fp, # 32
  difference_fp, # 35
  drfp, # 35
  esm1bts, # 42
  esm1b, # 40



]

esm1b results: {'pearsonr': 0.63, 'r2': 0.4, 'mse': 0.81}


## 2. Training a model with only sequence information (ESM-1b_ts):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python


from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [data_train["ESM1b_ts"]]
test_X = [data_test["ESM1b_ts"]]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


find_best_params(train_X, train_Y, test_X, test_Y, max_evals=3)

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [data_train["ESM1b_ts"]]
test_X = [data_test["ESM1b_ts"]]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.2831145406836757,
    "max_delta_step": 0.07686715986169101,
    "max_depth": 4.96836783761305,
    "min_child_weight": 6.905400087083855,
    "num_rounds": 313.1498988074061,
    "reg_alpha": 1.717314107718892,
    "reg_lambda": 2.470354543039016,
}

model_name = "esm1bts"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)



esm1bts results: {'pearsonr': 0.65, 'r2': 0.42, 'mse': 0.78}


## 3. Training a model with only reaction information (DRFP):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python


from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [data_train["DRFP"]]
test_X = [data_test["DRFP"]]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


find_best_params(train_X, train_Y, test_X, test_Y, max_evals=3)

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [data_train["DRFP"]]
test_X = [data_test["DRFP"]]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.08987247189322463,
    "max_delta_step": 1.1939737318908727,
    "max_depth": 11.268531225242574,
    "min_child_weight": 2.8172720953826302,
    "num_rounds": 400.03643430746544,
    "reg_alpha": 1.9412226989868904,
    "reg_lambda": 4.950543905603358,
}


model_name = "drfp"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)



drfp results: {'pearsonr': 0.61, 'r2': 0.35, 'mse': 0.87}


## 4. Training a model with only reaction information (difference fingerprint):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python


from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [data_train["difference_fp"]]
test_X = [data_test["difference_fp"]]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


find_best_params(train_X, train_Y, test_X, test_Y, max_evals=3)

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [data_train["difference_fp"]]
test_X = [data_test["difference_fp"]]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {'learning_rate': 0.14154883958006167,
         'max_delta_step': 0.02234358170535966,
         'max_depth': 10.869653004093198,
         'min_child_weight': 1.7936882442746056,
         'num_rounds': 361.6168542774665,
         'reg_alpha': 4.825525325323308,
         'reg_lambda': 2.74944090578774}


model_name = "difference_fp"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)



difference_fp results: {'pearsonr': 0.61, 'r2': 0.35, 'mse': 0.87}


## 5. Training a model with only reaction information (structural fingerprint):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python


from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [data_train["structural_fp"]]
test_X = [data_test["structural_fp"]]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


find_best_params(train_X, train_Y, test_X, test_Y, max_evals=3)

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [data_train["structural_fp"]]
test_X = [data_test["structural_fp"]]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {'learning_rate': 0.01126910440903659,
         'max_delta_step': 0.5777120839605732,
         'max_depth': 5.486901609313889,
         'min_child_weight': 6.14467742389769,
         'num_rounds': 488.943459090126,
         'reg_alpha': 4.629840853377147,
         'reg_lambda': 2.1047561335691745}


model_name = "structural_fp"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)



structural_fp results: {'pearsonr': 0.57, 'r2': 0.32, 'mse': 0.91}


## 6. Training a model with enzyme and reaction information (ESM1b_ts/DRFP):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python


from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)

train_X = [
  data_train["DRFP"],
  data_train["ESM1b_ts"]
  ]
test_X = [
  data_test["DRFP"],
  data_test["ESM1b_ts"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]



find_best_params(train_X, train_Y, test_X, test_Y, max_evals=3)

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
  data_train["DRFP"],
  data_train["ESM1b_ts"]
  ]
test_X = [
  data_test["DRFP"],
  data_test["ESM1b_ts"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param =  {
    'learning_rate': 0.0685249638804391,
    'max_depth': 9,
    'reg_lambda': 3.6231955480293934,
    'reg_alpha': 3.8923304126950957,
    'max_delta_step': 1.0371427260254895,
    'min_child_weight': 14.209819958496073,
    # 'sampling_method': 'gradient_based',
    'num_rounds': 400.9598325756988,
    }


model_name = "esm1bts_drfp"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)


XGBOOST training of esm1bts_drfp
esm1bts_drfp results: {'pearsonr': 0.66, 'r2': 0.43, 'mse': 0.76}


## 7. Training a model with enzyme and reaction information (ESM1b_ts/diff_fp):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python


from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)

train_X = [
  data_train["difference_fp"],
  data_train["ESM1b_ts"]
  ]
test_X = [
  data_test["difference_fp"],
  data_test["ESM1b_ts"]
  ]
train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


find_best_params(train_X, train_Y, test_X, test_Y, max_evals=3)

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
  data_train["difference_fp"],
  data_train["ESM1b_ts"]
  ]
test_X = [
  data_test["difference_fp"],
  data_test["ESM1b_ts"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    'learning_rate': 0.15638876027725748,
    'max_depth': 8,
    'reg_lambda': 2.344616112944763,
    'reg_alpha': 4.262441473730964,
    'max_delta_step': 0.9349225200130806,
    'min_child_weight': 8.26447488212503,
    'num_rounds': 400.69265795096726,
    # 'sampling_method': 'gradient_based'
    }

model_name = "esm1bts_diff"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)



esm1bts_diff results: {'pearsonr': 0.65, 'r2': 0.42, 'mse': 0.78}


## 8. Training a model with enzyme and reaction information (ESM1b_ts/DRFP/macc):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python


from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


print(data_train.columns)
train_X = [
  data_train["DRFP"],
  data_train["ESM1b_ts"],
  data_train["maccs_keys_combined"]
  ]
test_X = [
  data_test["DRFP"],
  data_test["ESM1b_ts"],
  data_test["maccs_keys_combined"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


find_best_params(train_X, train_Y, test_X, test_Y, max_evals=60)

Index(['Reaction ID', 'Sequence ID', 'kcat_values', 'Uniprot IDs',
       'from_BRENDA', 'from_Sabio', 'from_Uniprot', 'checked', 'Sequence',
       'substrates',
       ...
       'socn_normalized', 'qso_combined', 'qso_normalized', 'traid_combined',
       'traid_normalized', 'paac_combined', 'paac_normalized', 'ESM1b_norm',
       'ESM1b_ts_norm', 'log10_kcat_norm'],
      dtype='object', length=2302)
R2 score: 
0.010750660731321271
R2 score: 
0.1551008729796216
R2 score: 
0.20838833165906348
R2 score: 
0.2597314879709175
R2 score: 
0.29886487002010903
R2 score: 
0.25010611753248235
R2 score: 
0.007077072079097735
R2 score: 
-0.03149089770925513
R2 score: 
0.29662196608914887
R2 score: 
0.17426248757113585
R2 score: 
0.22695367151720056
R2 score: 
0.04748905901030744
R2 score: 
-0.08825834933874217
R2 score: 
-0.10465563078366599
R2 score: 
0.16249422785567064
R2 score: 
0.2727405476755494
R2 score: 
0.2623153045861992
R2 score: 
0.28448467515410036
R2 score: 
0.2054310721100971
R2 

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
    data_train["DRFP"],
    data_train["ESM1b_ts"],
    data_train["maccs_keys_combined"],
]
test_X = [data_test["DRFP"], data_test["ESM1b_ts"], data_test["maccs_keys_combined"]]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.09992412400920152,
    "max_delta_step": 3.1958455305363542,
    "max_depth": 8.746488189180099,
    "min_child_weight": 11.514644820660536,
    "num_rounds": 151.86858219483977,
    "reg_alpha": 3.2971597220525624,
    "reg_lambda": 3.8693000750961666,
}

model_name = "esm1bts_drfp_maccs"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)




esm1bts_drfp_maccs results: {'pearsonr': 0.64, 'r2': 0.4, 'mse': 0.8}


In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training

# Run the Python code and redirect output to a temporary file
python -u - <<END | tee /tmp/python_output.log
import time
for i in range(10):
  print(i)
  time.sleep(2)
END


0
1
2
3
4
5
6
7
8
9


## 9. Training a model with enzyme and reaction information (ESM1b_ts/DRFP/chemical_desc):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
# Run the Python code and redirect output to a temporary file
python -u - <<END | tee /tmp/python_output.log


from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


print(data_train.columns)
train_X = [
  data_train["DRFP"],
  data_train["ESM1b_ts"],
  data_train["chemical_descriptors_combined"]
  ]
test_X = [
  data_test["DRFP"],
  data_test["ESM1b_ts"],
  data_test["chemical_descriptors_combined"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

params_file = "xg_esm1bts_drfp_chem.txt"

find_best_params(train_X, train_Y, test_X, test_Y, params_file, max_evals=50)
END

Index(['Reaction ID', 'Sequence ID', 'kcat_values', 'Uniprot IDs',
       'from_BRENDA', 'from_Sabio', 'from_Uniprot', 'checked', 'Sequence',
       'substrates',
       ...
       'socn_normalized', 'qso_combined', 'qso_normalized', 'traid_combined',
       'traid_normalized', 'paac_combined', 'paac_normalized', 'ESM1b_norm',
       'ESM1b_ts_norm', 'log10_kcat_norm'],
      dtype='object', length=2302)
R2 score: 
0.3447344109986921
R2 score: 
0.19035910882037335
R2 score: 
-0.038092820838124306
R2 score: 
0.08614490557992724
R2 score: 
0.2338007974202579
R2 score: 
0.33461029879864407
R2 score: 
0.2818665679891583
R2 score: 
0.20269543069413185
R2 score: 
0.2822420025735456
R2 score: 
0.09924255434494031
R2 score: 
0.325531868056249
R2 score: 
0.3156116685378977
R2 score: 
0.04451614431804747
R2 score: 
0.09070185432049116
R2 score: 
0.015345158100477051
R2 score: 
0.2859325292939826
R2 score: 
0.023594438659029084
R2 score: 
0.2866846973691125
R2 score: 
0.007974678043599215
R2 scor

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
    data_train["DRFP"],
    data_train["ESM1b_ts"],
    data_train["chemical_descriptors_combined"],
]
test_X = [
    data_test["DRFP"],
    data_test["ESM1b_ts"],
    data_test["chemical_descriptors_combined"],
]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.13276400761169743,
    "max_delta_step": 3.001703956562063,
    "max_depth": 10.993562380573891,
    "min_child_weight": 3.557978119789838,
    "num_rounds": 104.58166183474756,
    "reg_alpha": 1.6844816534346863,
    "reg_lambda": 1.1912341257712378,
}

param = {'learning_rate': 0.21771422376635302, 'max_delta_step': 0.35901625428486683, 'max_depth': 7.929548188697309, 'min_child_weight': 14.817938859541384, 'num_rounds': 91.4648818100639, 'reg_alpha': 0.6649377315603816, 'reg_lambda': 0.40235312065442286}



model_name = "esm1bts_drfp_chem"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)





XGBOOST training of esm1bts_drfp_chem
esm1bts_drfp_chem results: {'pearsonr': 0.66, 'r2': 0.43, 'mse': 0.76}


## 10. Training a model with enzyme and reaction information (ESM1b_ts/DRFP/aa_composition):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python -u - <<END | tee /tmp/python_output.log


from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


print(data_train.columns)
train_X = [
  data_train["DRFP"],
  data_train["ESM1b_ts"],
  data_train["aa_composition_combined"]
  ]
test_X = [
  data_test["DRFP"],
  data_test["ESM1b_ts"],
  data_test["aa_composition_combined"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

params_file = "xg_esm1bts_drfp_aa_comp.txt"

find_best_params(train_X, train_Y, test_X, test_Y, params_file, max_evals=50)
END

Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
  File "/content/drive/.shortcut-targets-by-id/1iS6gSWfUE3cZnmrNWbbV9_W_zQH3vaiu/enz-eff-project/improved_code/model_training/xgboost_funs.py", line 1, in <module>
    from sklearn.metrics import r2_score, mean_squared_error
ModuleNotFoundError: No module named 'sklearn'


#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
    data_train["DRFP"],
    data_train["ESM1b_ts"],
    data_train["aa_composition_combined"],
]
test_X = [
    data_test["DRFP"],
    data_test["ESM1b_ts"],
    data_test["aa_composition_combined"],
]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.11721358463156084,
    "max_delta_step": 4.7633011783148635,
    "max_depth": 7.452086596583176,
    "min_child_weight": 6.7906316083399805,
    "num_rounds": 132.96203336778245,
    "reg_alpha": 3.6849748612212863,
    "reg_lambda": 3.4168451008462117,
}


model_name = "esm1bts_drfp_aa_composition"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)




XGBOOST training of esm1bts_drfp_aa_composition
esm1bts_drfp_aa_composition results: {'pearsonr': 0.64, 'r2': 0.41, 'mse': 0.79}


## 11. Training a model with enzyme and reaction information (ESM1b_ts/DRFP/dp_composition):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python -u - <<END | tee /tmp/python_output.log


from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


print(data_train.columns)
train_X = [
  data_train["DRFP"],
  data_train["ESM1b_ts"],
  data_train["dp_composition_combined"]
  ]
test_X = [
  data_test["DRFP"],
  data_test["ESM1b_ts"],
  data_test["dp_composition_combined"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

params_file = "xg_esm1bts_drfp_dp_comp.txt"

find_best_params(train_X, train_Y, test_X, test_Y, params_file, max_evals=50)



Index(['Reaction ID', 'Sequence ID', 'kcat_values', 'Uniprot IDs',
       'from_BRENDA', 'from_Sabio', 'from_Uniprot', 'checked', 'Sequence',
       'substrates',
       ...
       'socn_normalized', 'qso_combined', 'qso_normalized', 'traid_combined',
       'traid_normalized', 'paac_combined', 'paac_normalized', 'ESM1b_norm',
       'ESM1b_ts_norm', 'log10_kcat_norm'],
      dtype='object', length=2302)
R2 score: 
0.1092523206856418
R2 score: 
0.2788339031596038
R2 score: 
0.14068200055368857
R2 score: 
0.08062646762483942
R2 score: 
0.029092479738163556
R2 score: 
0.2465189769822876
R2 score: 
-0.13110518961188347
R2 score: 
0.3040933467616987
R2 score: 
0.1977810477568612
R2 score: 
0.29987529634151694
R2 score: 
0.06840478420552658
R2 score: 
0.2577458856952971
R2 score: 
0.08805965737533154
R2 score: 
-0.03294769098870085
R2 score: 
0.24734360170440378
R2 score: 
-0.12112250923308879
R2 score: 
0.2623672231370656
R2 score: 
0.2903580926485912
R2 score: 
0.2257885400441501
R2 score

bash: line 44: warning: here-document at line 3 delimited by end-of-file (wanted `END')


#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
    data_train["DRFP"],
    data_train["ESM1b_ts"],
    data_train["dp_composition_combined"],
]
test_X = [
    data_test["DRFP"],
    data_test["ESM1b_ts"],
    data_test["dp_composition_combined"],
]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.05500561350223968,
    "max_delta_step": 1.2238351648889834,
    "max_depth": 6.808389668454419,
    "min_child_weight": 1.1401741833363945,
    "num_rounds": 105.10391617103177,
    "reg_alpha": 1.4373077826405751,
    "reg_lambda": 1.0938850655053933,
}


model_name = "esm1bts_drfp_dp_composition_combined"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)



XGBOOST training of esm1bts_drfp_dp_composition_combined
esm1bts_drfp_dp_composition_combined results: {'pearsonr': 0.61, 'r2': 0.37, 'mse': 0.85}


## 12. Training a model with enzyme and reaction information (ESM1b_ts/DRFP/moreau_broto_auto_combined):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python -u - <<END | tee /tmp/python_output.log


from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


print(data_train.columns)
train_X = [
  data_train["DRFP"],
  data_train["ESM1b_ts"],
  data_train["moreau_broto_auto_combined"]
  ]
test_X = [
  data_test["DRFP"],
  data_test["ESM1b_ts"],
  data_test["moreau_broto_auto_combined"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


params_file = "xg_esm1bts_drfp_moreau_broto_auto.txt"

find_best_params(train_X, train_Y, test_X, test_Y, params_file, max_evals=50)



#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
    data_train["DRFP"],
    data_train["ESM1b_ts"],
    data_train["moreau_broto_auto_combined"],
]
test_X = [
    data_test["DRFP"],
    data_test["ESM1b_ts"],
    data_test["moreau_broto_auto_combined"],
]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.1038676163423828,
    "max_delta_step": 1.4548970171463687,
    "max_depth": 8.321249185474159,
    "min_child_weight": 6.806873952076018,
    "num_rounds": 121.80088819024851,
    "reg_alpha": 1.9147128067617487,
    "reg_lambda": 4.260510766683824,
}


model_name = "esm1bts_drfp_moreau_broto_auto_combined"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)




XGBOOST training of esm1bts_drfp_moreau_broto_auto_combined
esm1bts_drfp_moreau_broto_auto_combined results: {'pearsonr': 0.61, 'r2': 0.38, 'mse': 0.83}


## 13. Training a model with enzyme and reaction information (ESM1b_ts/DRFP/moran_auto_combined):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python -u - <<END | tee /tmp/python_output.log



from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


print(data_train.columns)
train_X = [
  data_train["DRFP"],
  data_train["ESM1b_ts"],
  data_train["moran_auto_combined"]
  ]
test_X = [
  data_test["DRFP"],
  data_test["ESM1b_ts"],
  data_test["moran_auto_combined"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


find_best_params(train_X, train_Y, test_X, test_Y, max_evals=50)
END

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
    data_train["DRFP"],
    data_train["ESM1b_ts"],
    data_train["moran_auto_combined"],
]
test_X = [data_test["DRFP"], data_test["ESM1b_ts"], data_test["moran_auto_combined"]]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.30651885762264497,
    "max_delta_step": 0.0648005181386535,
    "max_depth": 9.5090943201835,
    "min_child_weight": 1.6264621562212005,
    "num_rounds": 150.43633712507392,
    "reg_alpha": 2.4793161605837817,
    "reg_lambda": 4.189552265603117,
}


model_name = "esm1bts_drfp_moran_auto_combined"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)





XGBOOST training of esm1bts_drfp_moran_auto_combined
esm1bts_drfp_moran_auto_combined results: {'pearsonr': 0.64, 'r2': 0.4, 'mse': 0.8}


## 14. Training a model with enzyme and reaction information (ESM1b_ts/DRFP/geary_auto_combined):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python -u - <<END | tee /tmp/python_output.log



from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


print(data_train.columns)
train_X = [
  data_train["DRFP"],
  data_train["ESM1b_ts"],
  data_train["geary_auto_combined"]
  ]
test_X = [
  data_test["DRFP"],
  data_test["ESM1b_ts"],
  data_test["geary_auto_combined"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


params_file = "xg_esm1bts_drfp_geary_auto.txt"

find_best_params(train_X, train_Y, test_X, test_Y, params_file, max_evals=50)


Index(['Reaction ID', 'Sequence ID', 'kcat_values', 'Uniprot IDs',
       'from_BRENDA', 'from_Sabio', 'from_Uniprot', 'checked', 'Sequence',
       'substrates',
       ...
       'socn_normalized', 'qso_combined', 'qso_normalized', 'traid_combined',
       'traid_normalized', 'paac_combined', 'paac_normalized', 'ESM1b_norm',
       'ESM1b_ts_norm', 'log10_kcat_norm'],
      dtype='object', length=2302)
R2 score: 
0.07901512621596188
R2 score: 
0.25381970981456625
R2 score: 
0.21816813515791128
R2 score: 
-0.05190891641474229
R2 score: 
0.17348090327454582
R2 score: 
0.17164733597483414
R2 score: 
0.11916239951450944
R2 score: 
0.28235212610602123
R2 score: 
0.0038760225406826
R2 score: 
-0.041178146769232746
R2 score: 
0.1786092176822735
R2 score: 
0.24836752167949858
R2 score: 
0.27175768733943206
R2 score: 
0.2824669797727686
R2 score: 
0.16080596075956363
R2 score: 
0.2775891791417859
R2 score: 
0.0356783822405992
R2 score: 
0.2196555420251501
R2 score: 
0.05230585877974425
R2 sco

bash: line 45: warning: here-document at line 3 delimited by end-of-file (wanted `END')


#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
    data_train["DRFP"],
    data_train["ESM1b_ts"],
    data_train["geary_auto_combined"],
]
test_X = [data_test["DRFP"], data_test["ESM1b_ts"], data_test["geary_auto_combined"]]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.13760564003136985,
    "max_delta_step": 4.757620764861381,
    "max_depth": 4.217052496738367,
    "min_child_weight": 6.015888017784599,
    "num_rounds": 167.49738425267432,
    "reg_alpha": 3.526216871308287,
    "reg_lambda": 1.6794729375702733,
}


model_name = "esm1bts_drfp_geary_auto_combined"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)



XGBOOST training of esm1bts_drfp_geary_auto_combined
esm1bts_drfp_geary_auto_combined results: {'pearsonr': 0.61, 'r2': 0.37, 'mse': 0.85}


## 15. Training a model with enzyme and reaction information (ESM1b_ts/DRFP/ctd_combined):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python -u - <<END | tee /tmp/python_output.log



from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


print(data_train.columns)
train_X = [
  data_train["DRFP"],
  data_train["ESM1b_ts"],
  data_train["ctd_combined"]
  ]
test_X = [
  data_test["DRFP"],
  data_test["ESM1b_ts"],
  data_test["ctd_combined"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


find_best_params(train_X, train_Y, test_X, test_Y, max_evals=50)
END

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [data_train["DRFP"], data_train["ESM1b_ts"], data_train["ctd_combined"]]
test_X = [data_test["DRFP"], data_test["ESM1b_ts"], data_test["ctd_combined"]]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.2487586025848387,
    "max_delta_step": 0.21840365651520532,
    "max_depth": 10.209089887748997,
    "min_child_weight": 0.8614194077504848,
    "num_rounds": 97.65952860634053,
    "reg_alpha": 2.425317094413785,
    "reg_lambda": 1.938738158215626,
}


model_name = "esm1bts_drfp_ctd_combined"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)




XGBOOST training of esm1bts_drfp_ctd_combined
esm1bts_drfp_ctd_combined results: {'pearsonr': 0.63, 'r2': 0.39, 'mse': 0.81}


## 16. Training a model with enzyme and reaction information (ESM1b_ts/DRFP/paac_combined):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python -u - <<END | tee /tmp/python_output.log



from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


print(data_train.columns)
train_X = [
  data_train["DRFP"],
  data_train["ESM1b_ts"],
  data_train["paac_combined"]
  ]
test_X = [
  data_test["DRFP"],
  data_test["ESM1b_ts"],
  data_test["paac_combined"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


find_best_params(train_X, train_Y, test_X, test_Y, max_evals=50)


END

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [data_train["DRFP"], data_train["ESM1b_ts"], data_train["paac_combined"]]
test_X = [data_test["DRFP"], data_test["ESM1b_ts"], data_test["paac_combined"]]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.1364130916306728,
    "max_delta_step": 0.29972809555304514,
    "max_depth": 6.744610716801216,
    "min_child_weight": 6.408501810468082,
    "num_rounds": 185.34169261056266,
    "reg_alpha": 3.5558232162639154,
    "reg_lambda": 2.7120996219637084,
}


model_name = "esm1bts_drfp_paac_combined"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)




XGBOOST training of esm1bts_drfp_paac_combined
esm1bts_drfp_paac_combined results: {'pearsonr': 0.63, 'r2': 0.4, 'mse': 0.8}


## 17. Training a model with enzyme and reaction information (ESM1b_ts/DRFP/traid_combined):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python -u - <<END | tee /tmp/python_output.log



from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


print(data_train.columns)
train_X = [
  data_train["DRFP"],
  data_train["ESM1b_ts"],
  data_train["traid_combined"]
  ]
test_X = [
  data_test["DRFP"],
  data_test["ESM1b_ts"],
  data_test["traid_combined"]
  ]

train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


find_best_params(train_X, train_Y, test_X, test_Y, max_evals=50)
END

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [data_train["DRFP"], data_train["ESM1b_ts"], data_train["traid_combined"]]
test_X = [data_test["DRFP"], data_test["ESM1b_ts"], data_test["traid_combined"]]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.17531969167864045,
    "max_delta_step": 1.036422183087543,
    "max_depth": 6.147922131402633,
    "min_child_weight": 13.559843720255216,
    "num_rounds": 82.5628191610385,
    "reg_alpha": 1.1439877443708486,
    "reg_lambda": 1.8329374915637076,
}


model_name = "esm1bts_drfp_traid_combined"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)





XGBOOST training of esm1bts_drfp_traid_combined
esm1bts_drfp_traid_combined results: {'pearsonr': 0.64, 'r2': 0.4, 'mse': 0.8}


## 18. Training a model with enzyme and reaction information (ESM1b_ts/new_feats):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python -u - <<END | tee /tmp/python_output.log



from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
print(data_train.columns)

train_X = [
    data_train["ESM1b_ts"],
    data_train["maccs_keys_combined"],
    data_train["chemical_descriptors_combined"],
    data_train["aa_composition_combined"],
    data_train["dp_composition_combined"],
    data_train["moreau_broto_auto_combined"],
    data_train["moran_auto_combined"],
    data_train["geary_auto_combined"],
    data_train["ctd_combined"],
    data_train["paac_combined"],
    data_train["traid_combined"],
]

test_X = [
    data_test["ESM1b_ts"],
    data_test["maccs_keys_combined"],
    data_test["chemical_descriptors_combined"],
    data_test["aa_composition_combined"],
    data_test["dp_composition_combined"],
    data_test["moreau_broto_auto_combined"],
    data_test["moran_auto_combined"],
    data_test["geary_auto_combined"],
    data_test["ctd_combined"],
    data_test["paac_combined"],
    data_test["traid_combined"],
]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


params_file = "xg_esm1bts_all_feats.txt"

find_best_params(train_X, train_Y, test_X, test_Y, params_file, max_evals=50)
END

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
    data_train["ESM1b_ts"],
    data_train["maccs_keys_combined"],
    data_train["chemical_descriptors_combined"],
    data_train["aa_composition_combined"],
    data_train["dp_composition_combined"],
    data_train["moreau_broto_auto_combined"],
    data_train["moran_auto_combined"],
    data_train["geary_auto_combined"],
    data_train["ctd_combined"],
    data_train["paac_combined"],
    data_train["traid_combined"],
]

test_X = [
    data_test["ESM1b_ts"],
    data_test["maccs_keys_combined"],
    data_test["chemical_descriptors_combined"],
    data_test["aa_composition_combined"],
    data_test["dp_composition_combined"],
    data_test["moreau_broto_auto_combined"],
    data_test["moran_auto_combined"],
    data_test["geary_auto_combined"],
    data_test["ctd_combined"],
    data_test["paac_combined"],
    data_test["traid_combined"],
]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]

param = {
    "learning_rate": 0.050310919498210774,
    "max_delta_step": 4.3319939235068015,
    "max_depth": 7.386807960061238,
    "min_child_weight": 8.089511026239812,
    "num_rounds": 191.69825036370207,
    "reg_alpha": 4.573526295380227,
    "reg_lambda": 1.8133068017829785,
}


model_name = "esm1bts_all_new_feats"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)





XGBOOST training of esm1bts_all_new_feats
esm1bts_all_new_feats results: {'pearsonr': 0.65, 'r2': 0.41, 'mse': 0.79}


## 19. Training a model with enzyme and reaction information (ESM1b_ts*ESM1b/new_feats):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python -u - <<END | tee /tmp/python_output.log



from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
print(data_train.columns)

train_X = [
    data_train["ESM1b_ts"]*data_train["ESM1b"],
    data_train["maccs_keys_combined"],
    data_train["chemical_descriptors_combined"],
    data_train["aa_composition_combined"],
    data_train["dp_composition_combined"],
    data_train["moreau_broto_auto_combined"],
    data_train["moran_auto_combined"],
    data_train["geary_auto_combined"],
    data_train["ctd_combined"],
    data_train["paac_combined"],
    data_train["traid_combined"],
]

test_X = [
    data_test["ESM1b_ts"]*data_test["ESM1b"],
    data_test["maccs_keys_combined"],
    data_test["chemical_descriptors_combined"],
    data_test["aa_composition_combined"],
    data_test["dp_composition_combined"],
    data_test["moreau_broto_auto_combined"],
    data_test["moran_auto_combined"],
    data_test["geary_auto_combined"],
    data_test["ctd_combined"],
    data_test["paac_combined"],
    data_test["traid_combined"],
]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


params_file = "xg_esm1b_esm1bts_all_feats.txt"

find_best_params(train_X, train_Y, test_X, test_Y, params_file, max_evals=50)
END

Index(['Reaction ID', 'Sequence ID', 'kcat_values', 'Uniprot IDs',
       'from_BRENDA', 'from_Sabio', 'from_Uniprot', 'checked', 'Sequence',
       'substrates',
       ...
       'socn_normalized', 'qso_combined', 'qso_normalized', 'traid_combined',
       'traid_normalized', 'paac_combined', 'paac_normalized', 'ESM1b_norm',
       'ESM1b_ts_norm', 'log10_kcat_norm'],
      dtype='object', length=2302)
R2 score: 
0.2873819354275712
R2 score: 
-0.026539498082175907
R2 score: 
0.2167148871353933
R2 score: 
0.2946226565838456
R2 score: 
0.2835269675692717
R2 score: 
0.18994842355551397
R2 score: 
0.31778760679299584
R2 score: 
-0.023324023702778175
R2 score: 
0.05889494017565004
R2 score: 
0.29057007554576525
R2 score: 
0.20830463769674146
R2 score: 
-0.029023080478208296
R2 score: 
-0.018265812325035234
R2 score: 
0.002836361058849812
R2 score: 
0.33701963240344746
R2 score: 
-0.08876464164513362
R2 score: 
0.2537131021692807
R2 score: 
0.19043802008351424
R2 score: 
0.2397138039442956

#### (b) Training and validating model:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
    data_train["ESM1b_ts"] * data_train["ESM1b"],
    data_train["maccs_keys_combined"],
    data_train["chemical_descriptors_combined"],
    data_train["aa_composition_combined"],
    data_train["dp_composition_combined"],
    data_train["moreau_broto_auto_combined"],
    data_train["moran_auto_combined"],
    data_train["geary_auto_combined"],
    data_train["ctd_combined"],
    data_train["paac_combined"],
    data_train["traid_combined"],
]

test_X = [
    data_test["ESM1b_ts"] * data_test["ESM1b"],
    data_test["maccs_keys_combined"],
    data_test["chemical_descriptors_combined"],
    data_test["aa_composition_combined"],
    data_test["dp_composition_combined"],
    data_test["moreau_broto_auto_combined"],
    data_test["moran_auto_combined"],
    data_test["geary_auto_combined"],
    data_test["ctd_combined"],
    data_test["paac_combined"],
    data_test["traid_combined"],
]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


param = {
    "learning_rate": 0.10733137257229984,
    "max_delta_step": 2.1506214487681534,
    "max_depth": 5.327256510019331,
    "min_child_weight": 6.7213003584165625,
    "num_rounds": 195.96603536489076,
    "reg_alpha": 0.8975015244323159,
    "reg_lambda": 2.035710679749507,
}


model_name = "esm1bts_esm1b_combined_all_feats"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)




XGBOOST training of esm1bts_esm1b_combined_all_feats
esm1bts_esm1b_combined_all_feats results: {'pearsonr': 0.62, 'r2': 0.38, 'mse': 0.83}


## 20. Training a model with enzyme and reaction information (ESM1b_ts*ESM1b):

#### (a) Hyperparameter optimization:

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python -u - <<END | tee /tmp/python_output.log



from xgboost_funs import find_best_params
import numpy as np
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
print(data_train.columns)

train_X = [
    data_train["ESM1b_ts"]*data_train["ESM1b"],
]

test_X = [
    data_test["ESM1b_ts"]*data_test["ESM1b"],
]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


params_file = "xg_esm1b_esm1bts.txt"

find_best_params(train_X, train_Y, test_X, test_Y, params_file, max_evals=100)
END

Index(['Reaction ID', 'Sequence ID', 'kcat_values', 'Uniprot IDs',
       'from_BRENDA', 'from_Sabio', 'from_Uniprot', 'checked', 'Sequence',
       'substrates',
       ...
       'socn_normalized', 'qso_combined', 'qso_normalized', 'traid_combined',
       'traid_normalized', 'paac_combined', 'paac_normalized', 'ESM1b_norm',
       'ESM1b_ts_norm', 'log10_kcat_norm'],
      dtype='object', length=2302)
R2 score: 
0.12907047575798988
R2 score: 
0.25577627481615267
R2 score: 
-0.07475618349073945
R2 score: 
0.04679192498308291
R2 score: 
0.26629132225671315
R2 score: 
0.19112179374574906
R2 score: 
0.15927577000753396
R2 score: 
-0.016196699868077546
R2 score: 
0.27968726605532057
R2 score: 
0.2287871036555303
R2 score: 
0.3062740475949763
R2 score: 
0.12831343253883493
R2 score: 
0.05118464887363809
R2 score: 
0.12496278566391869
R2 score: 
0.2245865001681032
R2 score: 
0.11958214311301604
R2 score: 
0.3069437243309442
R2 score: 
0.04968308239245813
R2 score: 
0.25464231859259295
R2 s

#### (b) Training and validating model:

In [3]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python

from xgboost_funs import train_xgboost
import numpy as np
import pandas as pd
from os.path import join
import warnings

warnings.filterwarnings("ignore")


data_train = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl")
)
data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)


data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)


train_X = [
    data_train["ESM1b_ts"] * data_train["ESM1b"],
]

test_X = [
    data_test["ESM1b_ts"] * data_test["ESM1b"],
]


train_Y = data_train["log10_kcat"]
test_Y = data_test["log10_kcat"]


param = {
    "learning_rate": 0.07332193563988974,
    "max_delta_step": 4.341142499403841,
    "max_depth": 5.139364293216448,
    "min_child_weight": 8.246432022659084,
    "num_rounds": 186.72555889321325,
    "reg_alpha": 3.1095131693922964,
    "reg_lambda": 2.3336437518147,
}


model_name = "xg_esm1b_esm1bts"

train_xgboost(train_X, train_Y, test_X, test_Y, param, model_name)


XGBOOST training of xg_esm1b_esm1bts
xg_esm1b_esm1bts results: {'pearsonr': 0.61, 'r2': 0.38, 'mse': 0.83}


## Ensemble Xgboost

In [ ]:
%%bash
source activate envn
cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
python


from os.path import join
from xgboost_funs import infer_xgboost, calculate_weighted_mean, evaluate_model
import pandas as pd
import numpy as np


data_test = pd.read_pickle(
    join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl")
)

data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
test_Y = data_test["log10_kcat"]


test_X = [data_test["ESM1b"]]
esm1b_preds = infer_xgboost(test_X, test_Y, "esm1b.h5")

test_X = [data_test["ESM1b_ts"]]
esm1bts_preds = infer_xgboost(test_X, test_Y, "esm1bts.h5")

test_X = [data_test["DRFP"]]
drfp_preds = infer_xgboost(test_X, test_Y, "drfp.h5")

test_X = [data_test["difference_fp"]]
diff_preds = infer_xgboost(test_X, test_Y, "difference_fp.h5")

test_X = [data_test["structural_fp"]]
struct_preds = infer_xgboost(test_X, test_Y, "structural_fp.h5")

test_X = [data_test["difference_fp"], data_test["ESM1b_ts"]]
esm1bts_diff_preds = infer_xgboost(test_X, test_Y, "esm1bts_diff.h5")

test_X = [data_test["DRFP"], data_test["ESM1b_ts"]]
esm1bts_drfp_preds = infer_xgboost(test_X, test_Y, "esm1bts_drfp.h5")

test_X = [data_test["DRFP"], data_test["ESM1b_ts"], data_test["maccs_keys_combined"]]

esm1bts_drfp_maccs_preds = infer_xgboost(test_X, test_Y, "esm1bts_drfp_maccs.h5")


test_X = [
    data_test["DRFP"],
    data_test["ESM1b_ts"],
    data_test["chemical_descriptors_combined"],
]


esm1bts_drfp_chem_preds = infer_xgboost(test_X, test_Y, "esm1bts_drfp_chem.h5")


test_X = [
    data_test["DRFP"],
    data_test["ESM1b_ts"],
    data_test["aa_composition_combined"],
]


esm1bts_drfp_dp_composition_combined_preds = infer_xgboost(
    test_X, test_Y, "esm1bts_drfp_aa_composition.h5"
)


test_X = [
    data_test["DRFP"],
    data_test["ESM1b_ts"],
    data_test["dp_composition_combined"],
]

esm1bts_drfp_dp_composition_combined_preds = infer_xgboost(
    test_X, test_Y, "esm1bts_drfp_dp_composition_combined.h5"
)


test_X = [
    data_test["DRFP"],
    data_test["ESM1b_ts"],
    data_test["moreau_broto_auto_combined"],
]
esm1bts_drfp_moreau_broto_auto_combined_preds = infer_xgboost(
    test_X, test_Y, "esm1bts_drfp_moreau_broto_auto_combined.h5"
)


test_X = [data_test["DRFP"], data_test["ESM1b_ts"], data_test["moran_auto_combined"]]
esm1bts_drfp_moran_auto_combined_preds = infer_xgboost(
    test_X, test_Y, "esm1bts_drfp_moran_auto_combined.h5"
)


test_X = [data_test["DRFP"], data_test["ESM1b_ts"], data_test["geary_auto_combined"]]
esm1bts_drfp_geary_auto_combined_preds = infer_xgboost(
    test_X, test_Y, "esm1bts_drfp_geary_auto_combined.h5"
)


test_X = [data_test["DRFP"], data_test["ESM1b_ts"], data_test["ctd_combined"]]
esm1bts_drfp_ctd_combined_preds = infer_xgboost(
    test_X, test_Y, "esm1bts_drfp_ctd_combined.h5"
)


test_X = [data_test["DRFP"], data_test["ESM1b_ts"], data_test["paac_combined"]]
esm1bts_drfp_paac_combined_preds = infer_xgboost(
    test_X, test_Y, "esm1bts_drfp_paac_combined.h5"
)


test_X = [data_test["DRFP"], data_test["ESM1b_ts"], data_test["traid_combined"]]
esm1bts_drfp_traid_preds = infer_xgboost(
    test_X, test_Y, "esm1bts_drfp_traid_combined.h5"
)


test_X = [
    data_test["ESM1b_ts"],
    data_test["maccs_keys_combined"],
    data_test["chemical_descriptors_combined"],
    data_test["aa_composition_combined"],
    data_test["dp_composition_combined"],
    data_test["moreau_broto_auto_combined"],
    data_test["moran_auto_combined"],
    data_test["geary_auto_combined"],
    data_test["ctd_combined"],
    data_test["paac_combined"],
    data_test["traid_combined"],
]
esm1bts_all_new_feats_preds = infer_xgboost(test_X, test_Y, "esm1bts_all_new_feats.h5")


test_X = [
    data_test["ESM1b_ts"] * data_test["ESM1b"],
    data_test["maccs_keys_combined"],
    data_test["chemical_descriptors_combined"],
    data_test["aa_composition_combined"],
    data_test["dp_composition_combined"],
    data_test["moreau_broto_auto_combined"],
    data_test["moran_auto_combined"],
    data_test["geary_auto_combined"],
    data_test["ctd_combined"],
    data_test["paac_combined"],
    data_test["traid_combined"],
]

esm1bts_esm1b_combined_all_feats_preds = infer_xgboost(
    test_X, test_Y, "esm1bts_esm1b_combined_all_feats.h5"
)

combined_weighted_preds = np.array(
    [
        esm1b_preds,
        esm1bts_preds,
        drfp_preds,
        diff_preds,
        struct_preds,
        esm1bts_diff_preds,
        esm1bts_drfp_preds,
        esm1bts_esm1b_combined_all_feats_preds,
        esm1bts_all_new_feats_preds,
        esm1bts_drfp_traid_preds,
        esm1bts_drfp_paac_combined_preds,
        esm1bts_drfp_ctd_combined_preds,
        esm1bts_drfp_geary_auto_combined_preds,
        esm1bts_drfp_moran_auto_combined_preds,
        esm1bts_drfp_moreau_broto_auto_combined_preds,
        esm1bts_drfp_dp_composition_combined_preds,
        esm1bts_drfp_dp_composition_combined_preds,
        esm1bts_drfp_chem_preds,
        esm1bts_drfp_maccs_preds,
    ]
)

# Combine all predictions into a single numpy array
combined_weighted_preds = np.array(
    [
        esm1b_preds,  # esm1b.h5       # 40
        esm1bts_preds,  # esm1bts.h5    # 42
        drfp_preds,  # drfp.h5           # 35
        # diff_preds,  # difference_fp.h5 # 35
        # struct_preds,  # structural_fp.h5# 32
        esm1bts_diff_preds,  # esm1bts_diff.h5# 42
        esm1bts_drfp_preds,  # esm1bts_drfp.h5# 43
        # esm1bts_esm1b_combined_all_feats_preds,  # esm1bts_esm1b_combined_all_feats.h5# 38
        esm1bts_all_new_feats_preds,  # esm1bts_all_new_feats.h5# 41
        esm1bts_drfp_traid_preds,  # esm1bts_drfp_traid_combined.h5# 40
        esm1bts_drfp_paac_combined_preds,  # esm1bts_drfp_paac_combined.h5# 40
        # esm1bts_drfp_ctd_combined_preds,  # esm1bts_drfp_ctd_combined.h5# 39
        # esm1bts_drfp_geary_auto_combined_preds,  # esm1bts_drfp_geary_auto_combined.h5# 37
        esm1bts_drfp_moran_auto_combined_preds,  # esm1bts_drfp_moran_auto_combined.h5# 40
        # esm1bts_drfp_moreau_broto_auto_combined_preds,  # esm1bts_drfp_moreau_broto_auto_combined.h5# 38
        # esm1bts_drfp_dp_composition_combined_preds,  # esm1bts_drfp_dp_composition_combined.h5# 37
        esm1bts_drfp_chem_preds,  # esm1bts_drfp_chem.h5# 43
        esm1bts_drfp_maccs_preds,  # esm1bts_drfp_maccs.h5# 40
    ]
)

[
    "esm1bts_esm1b_combined_all_feats.h5",  # 38
    "esm1bts_all_new_feats.h5",  # 41
    "esm1bts_drfp_traid_combined.h5",  # 40
    "esm1bts_drfp_paac_combined.h5",  # 40
    "esm1bts_drfp_ctd_combined.h5",  # 39
    "esm1bts_drfp_geary_auto_combined.h5",  # 37
    "esm1bts_drfp_moran_auto_combined.h5",  # 40
    "esm1bts_drfp_moreau_broto_auto_combined.h5",  # 38
    "esm1bts_drfp_dp_composition_combined.h5",  # 37
    "esm1bts_drfp_aa_composition.h5",  # 41
    "esm1bts_drfp_chem.h5",  # 43,
    "esm1bts_drfp_maccs.h5",  # 40,
    "esm1bts_diff.h5",  # 42
    "esm1bts_drfp.h5", # 43
    "structural_fp.h5",  # 32
    "difference_fp.h5",  # 35
    "drfp.h5",  # 35
    "esm1bts.h5",  # 42
    "esm1b.h5",  # 40
]



weighted_avg_pred = calculate_weighted_mean(combined_weighted_preds, test_Y)
print(f"ensemble output: {evaluate_model(weighted_avg_pred, test_Y)}")


ensemble output: {'mse': 0.7, 'R2 score': 0.48, 'pearson coefficient': 0.69}


## Training model using all features

In [ ]:
train_indices = list(np.load(join("..", "..", "data", "kcat_data", "splits", "CV_train_indices.npy"), allow_pickle = True))
test_indices = list(np.load(join("..", "..", "data", "kcat_data", "splits", "CV_test_indices.npy"), allow_pickle = True))

In [ ]:
data_train = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_ours.pkl"))
data_test = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_ours.pkl"))

# print((train_indices))
# print((test_indices))

data_train.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
data_test.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
len(data_train), len(data_test)

(3391, 874)

In [ ]:
data_train.columns[:-7]

Index(['Reaction ID', 'Sequence ID', 'kcat_values', 'Uniprot IDs',
       'from_BRENDA', 'from_Sabio', 'from_Uniprot', 'checked', 'Sequence',
       'substrates',
       ...
       '765', '766', '767', '771', '772', '773', '774', '775', '776', '777'],
      dtype='object', length=2270)

In [ ]:
np.array(data_train['structural_fp'].tolist())[0]

array([1, 0, 0, ..., 0, 0, 0])

#### (a) Creating input matrices:

In [ ]:
train_X = np.array(list(data_train["ESM1b"]))
print(train_X.shape)
test_X = np.array(list(data_test["ESM1b"]))
for col in data_train.columns[17:21]:
  print(col)
  if col == 'structural_fp':
    train_X = np.concatenate([train_X, np.array(data_train[col].tolist(), dtype='float64')], axis = 1)
    test_X = np.concatenate([test_X, np.array(data_test[col].tolist(), dtype='float64')], axis = 1)
  else:
    train_X = np.concatenate([train_X, np.array(list(data_train[col]), dtype='float64')], axis = 1)
    test_X = np.concatenate([test_X, np.array(list(data_test[col]), dtype='float64')], axis = 1)

(3391, 1280)
ESM1b_ts
structural_fp
difference_fp
DRFP


In [ ]:
train_X = np.concatenate([train_X, np.array(list(data_train['ESM1b_norm']), dtype='float64')], axis = 1)
test_X = np.concatenate([test_X, np.array(list(data_test['ESM1b_norm']), dtype='float64')], axis = 1)
train_X = np.concatenate([train_X, np.array(list(data_train['ESM1b_ts_norm']), dtype='float64')], axis = 1)
test_X = np.concatenate([test_X, np.array(list(data_test['ESM1b_ts_norm']), dtype='float64')], axis = 1)

In [ ]:
train_X = np.concatenate([train_X, np.array(data_train.iloc[:,22:-7], dtype='float64')], axis = 1)
test_X = np.concatenate([test_X, np.array(data_test.iloc[:,22:-7], dtype='float64')], axis = 1)

In [ ]:
train_X = train_X.astype('float64')
test_X = test_X.astype('float64')

In [ ]:
train_Y = np.array(list(data_train["log10_kcat"]))
test_Y = np.array(list(data_test["log10_kcat"]))

In [ ]:
train_X.shape

(3391, 15560)

In [ ]:
type(train_X)

numpy.ndarray

#### (b) Hyperparameter optimization:

In [ ]:
def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)

 87%|████████▋ | 174/200 [5:08:10<40:19, 93.06s/trial, best loss: -0.3735840090139679]

In [ ]:
best

#### (c) Training and validating model:

In [ ]:
param = {
    'learning_rate': 0.15638876027725748,
    'max_depth': 8,
    'reg_lambda': 2.344616112944763,
    'reg_alpha': 4.262441473730964,
    'max_delta_step': 0.9349225200130806,
    'min_child_weight': 8.26447488212503,
    'num_rounds': 400.69265795096726,
    # 'sampling_method': 'gradient_based'
    }

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:

R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_all.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_all.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_all.npy"), np.array(R2))


[0.6133773796060704, 0.5927733649751115, 0.5696579476183089, 0.5664901288879961, 0.6059675908688378]
[0.8995350132420895, 0.8907148311609805, 0.9570658132750683, 1.0011614220188685, 0.9055139379678919]
[0.37289620835138915, 0.3481887641919241, 0.3220370010024399, 0.31963431757814464, 0.3656110621769998]


In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)


bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_all.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_all.npy"), test_Y)

save_pickel_model(bst, "xgboost_all.h5")

0.613 0.835 0.375


In [ ]:
y_test_pred_esm1b_ts_drfp = y_test_pred